In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scipy.io as sio
import anndata as ad
import seaborn as sns
import os as os
import sys as sys
sys.path.append('/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *
import operator as op
import matplotlib.colors as mcolors
data_dir_NHDP = "/home/qiuaodon/Desktop/project_data_new/"

# load in sc data

In [ ]:
data_dir_NHDP = "/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/"
adata_T = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_T_cells.h5ad')
adata_1 = adata_T[adata_T[:,'PDCD1'].X > 0, :]
adata_M = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_M_cells.h5ad')
adata_2 = adata_M

In [4]:
colors = ["grey", "blue"]  # Start with grey and end with blue
cmap = mcolors.LinearSegmentedColormap.from_list("grey_to_blue", colors)


In [6]:
adata_1_pseudo = scRNA2PseudoBulkAnnData(adata_1, sample_id_col='sample_id')
adata_2_pseudo = scRNA2PseudoBulkAnnData(adata_2, sample_id_col='sample_id')

# prepare data matrix for IVtest

In [ ]:
DEG_1 = paird_ttest(adata_1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')
DEG_2 = paird_ttest(adata_2, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')

In [7]:
DEG_1 = DEG_1[DEG_1['pval'] < 0.05]
DEG_2 = DEG_2[DEG_2['pval'] < 0.05]
gene_1 = DEG_1.index.tolist()
gene_2 = DEG_2.index.tolist()

# Filter out invalid gene names
gene_1 = [gene for gene in gene_1 if gene in adata_1_pseudo.var_names]
gene_2 = [gene for gene in gene_2 if gene in adata_2_pseudo.var_names]

gene_1_matrix = adata_1_pseudo[:, gene_1].X
gene_2_matrix = adata_2_pseudo[:, gene_2].X
gene_1_df = pd.DataFrame(gene_1_matrix, columns=gene_1, index=adata_1_pseudo.obs['sample_id'])
gene_2_df = pd.DataFrame(gene_2_matrix, columns=gene_2, index=adata_2_pseudo.obs['sample_id'])
gene_1_df.columns = [i + '_T' for i in gene_1_df.columns]
gene_2_df.columns = [i + '_M' for i in gene_2_df.columns]
gene_df = gene_1_df
gene_df = pd.merge(gene_df, gene_2_df, on='sample_id')

In [ ]:
gene_df['treatment'] = gene_df.index.str.contains('On').astype(int)
# get patient id from sample id remove the _On or _Pre
gene_df['patient_id'] = gene_df.index.str.replace('_On', '').str.replace('_Pre', '')
gene_df['treatment'].value_counts()

In [38]:
# export the gene_df to csv
# gene_df.to_csv(data_dir_NHDP + 'gene_df_PD1_Myeloid.csv')
gene_df.to_csv(data_dir_NHDP + 'gene_df_PD1_Myeloid_GEM.csv')

# load in IV result and perform CIT

In [9]:
# DEG_pairs = pd.read_excel('/home/qiuaodon/Desktop/project_data_new/IV_regression_results_PD1vsM.xlsx')
DEG_pairs = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/IV_regression_results_PD1vsM_including_qval.csv')
# DEG_pairs = pd.read_excel('/home/qiuaodon/Desktop/project_data_new/IV_regression_results_CD4EXvsEndo_filtered.xlsx')
# drop the p_value column and r_squared column
# DEG_pairs = DEG_pairs.drop(columns=['p_value', 'r_squared'])
# set the first two columns as g1 and g2 in DEG_pairs
DEG_pairs = DEG_pairs.rename(columns = {'gene_T':'g1', 'gene_cell':'g2'})

In [ ]:

calculate_gene_correlation(DEG_pairs, adata_1_pseudo, adata_2_pseudo)

In [ ]:
# Combine the two conditions using the logical OR operator
DEG_pairs = DEG_pairs[(DEG_pairs['g1vsg2_correlation'] > 0.3) | (DEG_pairs['g1vsg2_correlation'] < -0.3)]
DEG_pairs = DEG_pairs[(DEG_pairs['g1vsg2_correlation'] > 0.3)]
# sort the DEG_pairs by g1vsg2_correlation
DEG_pairs = DEG_pairs.sort_values(by='g1vsg2_correlation', ascending=False)
DEG_pairs

In [ ]:
g1 = list(DEG_pairs['g1']) 
g2 = list(DEG_pairs['g2']) 
g1_in_CD4EX = [i for i in gene_df.columns if np.sum(gene_df.loc[:, i] == 0) < 15] 
g2_in_Endo = [i for i in gene_df.columns if np.sum(gene_df.loc[:, i] == 0) < 15] 
DEG_pairs = DEG_pairs[DEG_pairs['g1'].isin(g1_in_CD4EX) & DEG_pairs['g2'].isin(g2_in_Endo)] 
DEG_pairs 

In [ ]:
lrpair = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/lr_network_unique.tsv', sep='\t')
# get the lr pairs including ligand(from) and receptor(to) from the lrpair file with database as 'kegg'
lrpair = lrpair[['from', 'to']]
# change from as L and to as R
lrpair.columns = ['L', 'R']
lrpair

In [ ]:
# check if the L and R in adata_CD4EX and adata_Mono separately and expression not all zero
L = list(lrpair['L'])
R = list(lrpair['R'])
L_in_1 = list(set(L).intersection(adata_1.var_names))
R_in_2 = list(set(R).intersection(adata_2.var_names))
L_in_1 = [x for x in L_in_1 if np.sum(adata_1_pseudo[:, x].X == 0) < 0.75 * adata_1_pseudo.shape[0]]
R_in_2 = [x for x in R_in_2 if np.sum(adata_2_pseudo[:, x].X == 0) < 0.75 * adata_2_pseudo.shape[0]]
print('L_in_CD4EX:', len(L_in_1))
print('R_in_2:', len(R_in_2))

In [ ]:
# remove the LR pairs not in L_in_CD4EX and R_in_DC
lrpair = lrpair[lrpair['L'].isin(L_in_1)]
lrpair = lrpair[lrpair['R'].isin(R_in_2)]
print('lrpair_kegg:', lrpair.shape)

In [12]:
# check if CXCL13-ACKR1 in the LR_pairs
lrpair[(lrpair['L'] == 'CXCL13') & (lrpair['R'] == 'ACKR1')]
new_pair = pd.DataFrame({'L': ['CXCL13'], 'R': ['ACKR1']})
lrpair = pd.concat([lrpair, new_pair], ignore_index=True)

# CIT using fisher-z

In [22]:
def CIT_test(DEG_pairs, lrpairs, adata_1_pseudo, adata_2_pseudo, gene_df, method="kci", p_value_threshold=0.05):
    results = pd.DataFrame(columns=['g1', 'g2', 'L', 'R', 'pValue'])

    for index, row in DEG_pairs.iterrows():
        g1 = row['g1']
        g2 = row['g2']
        for L, R in lrpairs.values:
                # Check if L and R can be found in the DEG expression data
                if L in adata_1_pseudo.var_names and R in adata_2_pseudo.var_names:
                    # Get the L_T and R_B from pseudo_T and pseudo_M
                    L_T = adata_1_pseudo[:, L].X
                    L_T = pd.DataFrame(L_T, columns=[L], index=adata_1_pseudo.obs['sample_id']).dropna()
                    R_M = adata_2_pseudo[:, R].X
                    R_M = pd.DataFrame(R_M, columns=[R], index=adata_2_pseudo.obs['sample_id']).dropna()
                    
                    # Calculate L*R
                    L_T.columns = [R + '_vs_' + L]
                    R_M.columns = L_T.columns
                    L_R = L_T.multiply(R_M, axis=0).iloc[:, 0]

                    # Make a DataFrame g1, g2 and L*R
                    if g1 not in gene_df.columns or g2 not in gene_df.columns:
                        continue
                    
                    g1_T = gene_df[g1]
                    g2_M = gene_df[g2]
                    df = pd.DataFrame({g1: g1_T, g2: g2_M, 'L*R': L_R}).dropna()

                    # Convert df to numpy array
                    df_numpy = df.to_numpy()

                    # Perform the chosen independence test
                    if method == "kci":
                        cit_obj = CIT(df_numpy, "kci")
                    elif method == "fisherz":
                        cit_obj = CIT(df_numpy, "fisherz")
                    else:
                        raise ValueError("Unsupported method. Use 'kci' or 'fisherz'.")
                    
                    pValue = cit_obj(0, 1, [2])
                    results = pd.concat([results, pd.DataFrame({'g1': [g1], 'g2': [g2], 'L': [L], 'R': [R], 'pValue': [pValue]})], ignore_index=True)

    return results

In [23]:
results = CIT_test(DEG_pairs, lrpair, adata_1_pseudo, adata_2_pseudo, gene_df, method = 'fisherz', p_value_threshold=0.05)

In [ ]:
#calculate the correlation between g1 and g2
calculate_gene_correlation(results, adata_1_pseudo, adata_2_pseudo)
calculate_residuals_correlation(results, adata_1_pseudo, adata_2_pseudo)

In [ ]:
# save the results to csv
results.to_csv(data_dir_NHDP + 'IT_test_results_PD1_Myeloid_alCl_results.csv')

## draw heatmap

In [ ]:
# sort DEG1_1 and DEG1_2 by the p_value
DEG_1 = DEG_1.sort_values(by='pval')
DEG_2 = DEG_2.sort_values(by='pval')
DEG_1

In [7]:
# top 20 genes in DEG_1 and DEG_2 as top_genes_T and top_genes_M
top_genes_T = DEG_1.index[:25]
top_genes_M = DEG_2.index[:25]


In [ ]:
# replace the ETS2 in top_genes_M with RHOB
top_genes_M = top_genes_M.drop('ETS2')
top_genes_M = top_genes_M.append(pd.Index(['RHOB']))
top_genes_M

In [ ]:
top_genes_T

In [13]:
top_genes_M = [
    'TSC22D3', 'DDIT4', 'FKBP5', 'SLC1A3', 'AREG', 'ACSL1',
    'RGS1', 'CH25H', 'SMIM3', 'TUBA1A', 'HERPUD1','RNF144B', 'KCNE1', 'TENT5A',
    'SLC19A2', 'SOCS1', 'DUSP6', 'TNFAIP3', 'PFKFB3', 'RHOB', 'SESN1', 'PDK4',
    'ARRDC2', 'MZF1-AS1', 'CEBPD'
]


In [ ]:
top_genes_T = [
    'CH25H',
    'TSC22D3',
    'TNFAIP3',
    'BIRC3',
    'SPON2',
    'ZFP36',
    'SLA',
    'ZFP36',
    'TXNIP',
    'IER2',
    'KLF10'
]

In [ ]:
import seaborn as sns
import scipy.stats as stats
from scipy.stats import spearmanr 
import numpy as np

# Your existing data preparation code remains the same
adata_T_pseudo = adata_1_pseudo
adata_M_pseudo = adata_2_pseudo
adata_M_pseudo = adata_M_pseudo[adata_M_pseudo.obs['sample_id'].isin(adata_T_pseudo.obs['sample_id'])]
adata_T_pseudo = adata_T_pseudo[adata_T_pseudo.obs['sample_id'].isin(adata_M_pseudo.obs['sample_id'])]

# Initialize matrices
corr_matrix = np.zeros((len(top_genes_T), len(top_genes_M)))
pval_matrix = np.zeros((len(top_genes_T), len(top_genes_M)))

# Calculate correlations
for i, gene_T in enumerate(top_genes_T):
    for j, gene_M in enumerate(top_genes_M):
        gene_T = gene_T.replace('_T', '')
        gene_M = gene_M.replace('_M', '')
        gene_T_expression = adata_T_pseudo[:, adata_T_pseudo.var_names == gene_T].X.flatten()
        gene_M_expression = adata_M_pseudo[:, adata_M_pseudo.var_names == gene_M].X.flatten()
        corr, pval = spearmanr(gene_T_expression, gene_M_expression)
        corr_matrix[i, j] = corr
        pval_matrix[i, j] = -np.log10(pval)

# Sort the correlation matrix
# First, calculate the mean correlation for each row and column
row_means = np.mean(corr_matrix, axis=1)
col_means = np.mean(corr_matrix, axis=0)

# Get sorting indices
row_idx = np.argsort(row_means)[::-1]  # Sort in descending order
col_idx = np.argsort(col_means)[::-1]  # Sort in descending order

# Sort the matrices and gene names
corr_matrix_sorted = corr_matrix[row_idx][:, col_idx]
pval_matrix_sorted = pval_matrix[row_idx][:, col_idx]
top_genes_T_sorted = np.array(top_genes_T)[row_idx]
top_genes_M_sorted = np.array(top_genes_M)[col_idx]

# Plot the sorted correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix_sorted, annot=False, cmap='coolwarm', vmin=-1, vmax=1,
            xticklabels=top_genes_M_sorted, yticklabels=top_genes_T_sorted)

# Customize tick labels
plt.gca().set_xticklabels(top_genes_M_sorted, fontstyle='italic', rotation=45, ha='right')
plt.gca().set_yticklabels(top_genes_T_sorted, fontstyle='italic', rotation=0)
plt.title('Correlation between Highly Correlated DEGs in T and M cells\n')
plt.xlabel('')
plt.ylabel('')
plt.tight_layout()  # Added to prevent label cutoff
plt.show()

#### plot the scatter plots


In [19]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import spearmanr, pearsonr

def plot_CIT_DEGcorr(g1, g2, L, R, DEG_exp, pseudo_1, pseudo_2):
    # Define a custom palette
    palette = {'pre': "#E69F00", 'on': "#56B4E9"}
    #replace the treatment column 0 pre and 1 on
    gene_df['treatment'] = gene_df['treatment'].replace({0: 'pre', 1: 'on'})
    # Set font sizes
    title_fontsize = 14
    axis_fontsize = 14
    legend_fontsize = 12

    def adjust_plot_limits(ax, x_data, y_data):
        x_min, x_max = x_data.min(), x_data.max()
        y_min, y_max = y_data.min(), y_data.max()
        x_padding = (x_max - x_min) * 0.1
        y_padding = (y_max - y_min) * 0.1
        ax.set_xlim(x_min - x_padding, x_max + x_padding)
        ax.set_ylim(y_min - y_padding, y_max + y_padding)

    # Plot the jointplot between DEG of g1 and g2
    g1_exp = DEG_exp[g1]
    g2_exp = DEG_exp[g2]
    df1 = pd.DataFrame({g1: g1_exp, g2: g2_exp, 'treatment': DEG_exp['treatment']})
    df1 = df1.dropna()
    g = sns.jointplot(x=g1, y=g2, data=df1, hue='treatment', palette=palette, kind='scatter', marginal_kws=dict(fill=True))
    sns.regplot(x=g1, y=g2, data=df1, scatter=False, ax=g.ax_joint, color='black')
    adjust_plot_limits(g.ax_joint, df1[g1], df1[g2])
    
    g.ax_joint.tick_params(left=False, bottom=False)
    g.ax_marg_x.tick_params(bottom=False)
    g.ax_marg_y.tick_params(left=False)
    cor = pearsonr(df1[g1], df1[g2])[0]
    plt.suptitle(f'Correlation between {g1} and {g2} is {cor:.3f}', fontsize=title_fontsize)
    plt.subplots_adjust(top=0.95)
    plt.show()

     # Plot the jointplot between DEG of g1 and g2 residue
    L_values = pseudo_1[:, L].X.toarray().flatten()
    R_values = pseudo_2[:, R].X.toarray().flatten()
    L_R = L_values * R_values
    X = sm.add_constant(L_R)

    # Fit OLS model for g2
    model_g2 = sm.OLS(g2_exp, X).fit()
    residuals_g2 = model_g2.resid

    # Fit OLS model for g1
    model_g1 = sm.OLS(g1_exp, X).fit()
    residuals_g1 = model_g1.resid

    #   Create DataFrame for residuals
    df_res = pd.DataFrame({
        'G_1_residual': residuals_g1,
        'G_2_residual': residuals_g2,
        'treatment': DEG_exp['treatment']
    })

    # Plot the residuals
    g = sns.jointplot(x='G_1_residual', y='G_2_residual', data=df_res, hue='treatment', palette=palette, kind='scatter', marginal_kws=dict(fill=True))
    sns.regplot(x='G_1_residual', y='G_2_residual', data=df_res, scatter=False, ax=g.ax_joint, color='black')

    # Adjust plot limits if needed
    adjust_plot_limits(g.ax_joint, df_res['G_1_residual'], df_res['G_2_residual'])

    # Customize plot appearance
    g.ax_joint.tick_params(left=False, bottom=False)
    g.ax_marg_x.tick_params(bottom=False)
    g.ax_marg_y.tick_params(left=False)

    # Calculate and display correlation
    corr_res = pearsonr(df_res['G_1_residual'], df_res['G_2_residual'])[0]
    plt.suptitle(f'Correlation between residuals of G_1 and G_2 is {corr_res:.3f}', fontsize=title_fontsize)
    plt.subplots_adjust(top=0.95)
    plt.show()



# check the pvalue of GEM_T_10 and GEM_M_10 corr

In [17]:
import numpy as np

# Define gene sets
GEM_T_10 = ['CXCR4', 'IL7R', 'TSC22D3', 'TXNIP', 'ZFP36', 'ZFP36L2']
GEM_M_10 = ['ACSL1', 'ARRDC3', 'CD163', 'CPM', 'ELL2', 'GLUL', 'MAFB', 'NLRP3', 'PFKFB3', 'PLIN2', 'RCAN1', 'RHOB', 'SESN1']

# Function to calculate average expression and add to .obs
def add_avg_expression(adata, gene_list, colname):
    genes_present = [g for g in gene_list if g in adata.var_names]
    avg_expr = np.asarray(adata[:, genes_present].X.mean(axis=1)).ravel()
    adata.obs[colname] = avg_expr

# Apply to adata_1_pseudo
add_avg_expression(adata_1_pseudo, GEM_T_10, 'GEM_T_10')
add_avg_expression(adata_1_pseudo, GEM_M_10, 'GEM_M_10')

# Apply to adata_2_pseudo
add_avg_expression(adata_2_pseudo, GEM_T_10, 'GEM_T_10')
add_avg_expression(adata_2_pseudo, GEM_M_10, 'GEM_M_10')


In [ ]:
plot_CIT_DEGcorr('GEM_T_10', 'GEM_M_10', 'ITGA4', 'TFRC', gene_df, adata_1_pseudo, adata_2_pseudo)